# Multi-Agent Systems in ADK

**Use Case:** When one agent isn’t enough. Use multiple specialized agents that talk to each other.

Google ADK supports **multi-agent conversations** via the `MultiAgent` class.
You can define:
- A list of `agents`
- A `selector` to decide who speaks next
- An optional `role_assignment` to change names/roles


### How MultiAgent works

```mermaid
flowchart TD
  U[User Message] --> R(Runner)
  R --> MA[MultiAgent]
  MA -->|selects| A1[Agent 1]
  MA --> A2[Agent 2]
  A1 --> MA
  A2 --> MA
  MA --> R --> U
```

- All agents share one session and state.
- The `selector` decides which agent speaks at each step.
- The conversation continues until one agent returns `is_final_response()`.

### Roles: name override
You can assign display roles to agents like `"Researcher"`, `"Critic"`, etc.

```python
role_assignment = {
  agent1.name: "Researcher",
  agent2.name: "Summarizer"
}
```

### Selector types
- `RoundRobinSelector` → simple back-and-forth
- `ContentBasedSelector` → uses LLM to choose next agent
- `MaxTurnsSelector(N)` → ends after N total turns

These selectors are in `google.adk.selectors`.

### Example: Research + Summary
Agent 1: Finds facts.  
Agent 2: Summarizes those facts into a short answer.

**Goal:** Show collaboration using `RoundRobinSelector`.

In [ ]:
from google.adk.agents import LlmAgent, MultiAgent
from google.adk.selectors import RoundRobinSelector
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

agent1 = LlmAgent(
    model="gemini-2.5-flash",
    name="fact_agent",
    instruction="List 3 quick facts about the topic."
)

agent2 = LlmAgent(
    model="gemini-2.5-flash",
    name="summary_agent",
    instruction="Summarize the above facts in 1 short paragraph."
)

multi = MultiAgent(
    agents=[agent1, agent2],
    selector=RoundRobinSelector(),
    role_assignment={
        "fact_agent": "Researcher",
        "summary_agent": "Summarizer"
    }
)

session_service = InMemorySessionService()
session_service.create_session("multiapp", "u1", "s1")

runner = Runner(agent=multi, app_name="multiapp", session_service=session_service)
prompt = Content(role="user", parts=[Part(text="Tell me about electric cars")])

for e in runner.run("u1", "s1", new_message=prompt):
    if e.is_final_response():
        print("\nFinal Answer:\n", e.content.parts[0].text)

### Notes
- Use **shared state** to pass info between agents (`output_key`, `state_delta`)
- `MultiAgent` can run **any ADK agents**: LLM agents, tool agents, etc.
- Keep roles + selectors simple for now. Add tools only once this flow works.

### Selector with max turns
```python
from google.adk.selectors import MaxTurnsSelector

multi = MultiAgent(
    agents=[agent1, agent2],
    selector=MaxTurnsSelector(max_turns=4)
)
```

Useful when you want a controlled exchange between agents before finishing.

---
### Troubleshooting
- ✅ **Only one agent replying?** → Check selector logic.
- ✅ **Final text missing?** → Make sure one agent calls `is_final_response()` via a final event.
- ✅ **Confused roles?** → Use `role_assignment` to label agents clearly.

---
### Summary
- Use `MultiAgent()` when multiple agents are needed.
- Add agents, selector, and (optional) role assignments.
- All agents share the same `session` and `state`.
- Works with Runner and Web UI just like single agents.

> Next: Try this in `adk web .` and observe how agents take turns in the **Events** tab.